In [28]:
import math as m # math library
from scipy.spatial import distance
import numpy as np
import scipy as sp
import pandas as pd

import plotly
plotly.tools.set_credentials_file(username='agu3rra', api_key='b6WWbmeqKfYxPkQJBnw8') # setting up credentials; Plotly is an online service.
import plotly.plotly as py # import graphics library
import plotly.graph_objs as go

# ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # -----
# Global variables
a = 6378137.0 # semi-major axis a using WGS84 standard (World Geodetic System)
b = 6356752.314245 # semi-minor axis b using WGS84 standard
f = (a-b)/a # first flattening of the Earth taken into account
e2 = 2*f-m.pow(f,2) # e squared parameter

# ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # -----
# Functions
def deg2rad (degrees):
    # Converts from degrees to radians
    return degrees*m.pi/180

def rad2deg (radians):
    # Converts from radians to degrees
    return radians*180/m.pi

def calculateV (lat):
    # Return v for the given latitude in degrees
    latRad = deg2rad(lat) # converts to radians
    return a/m.sqrt(1-e2*m.pow(m.sin(latRad),2))

def geo2car (lat,lon,ele):
    # Returns point in Cartesian coordinates given its Geographic coordinates
    # Data returned in Pandas Series form

    v = calculateV(lat)

    latRad = deg2rad(lat)
    lonRad = deg2rad(lon)
    z = (v*(1-e2)+ele)*m.sin(latRad)
    y = (v+ele)*m.cos(latRad)*m.sin(lonRad)
    x = (v+ele)*m.cos(latRad)*m.cos(lonRad)

    pointC = pd.Series(dict(x=x, y=y, z=z))
    return pointC # returns point in Cartesian coordinates

# ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # -----
# Test code for distance measurements and conversion from Geographic to
# Cartesian Coordinates

# pa = geo2cat(-24.9361993186,-51.3906013593,883.16)
# pb = geo2cat(-24.9356264155,-51.3912779465,882.69)
#
# dst = distance.euclidean(pa,pb)
# print(dst)
# print(pa)
# print(pb)

# ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # ----- # -----
# MAIN CODE

# Reading data from CSV file and placing it on a numpy array
dfGPS = pd.read_csv('gpsData.csv', sep=',') # import csv data into pandas dataframe

In [19]:
x = dfGPS[['latitude']].values
y = dfGPS[['longitude']].values
z = dfGPS[['elevation']].values

In [21]:
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~agu3rra/0 or inside your plot.ly account where it is named 'simple-3d-scatter'


In [32]:
dfCartesian = pd.DataFrame([]) # initialize new dataframe
for rowIndex, row in dfGPS.iterrows(): #Iterating thru elements in dataframe
    pointCartesian = geo2car(row['latitude'],row['longitude'],row['elevation']) # convert to cartesian coordinates
    dfCartesian = dfCartesian.append(pointCartesian, ignore_index=True) # append newly converted data to dataframe
dfCartesian.to_csv('gpsDataCartesian.csv',index=False) # save to file

In [33]:
x = dfCartesian[['x']].values
y = dfCartesian[['y']].values
z = dfCartesian[['z']].values

In [35]:
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=5,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~agu3rra/0 or inside your plot.ly account where it is named 'simple-3d-scatter'


In [37]:
data = [
    go.Contour(
        z=z,
        x=x,
        y=y
    )]

py.iplot(data)

In [38]:
x = dfCartesian[['x']].values
y = dfCartesian[['y']].values
z = dfGPS[['elevation']].values

In [39]:
data = [
    go.Contour(
        z=z,
        x=x,
        y=y
    )]

py.iplot(data)

In [40]:
x = dfGPS[['latitude']].values
y = dfGPS[['longitude']].values
z = dfGPS[['elevation']].values

In [41]:
data = [
    go.Contour(
        z=z,
        x=x,
        y=y
    )]

py.iplot(data)

In [42]:
import utm
utm.from_latlon(-24.9361993186,-51.3906013593,883.16)

(-226727318209172.47, -6.116223956084173e+16, 883.16, 'J')

In [43]:
utm.from_latlon(-24.9361993186,-51.3906013593)

(460564.002392906, 7242060.106369796, 22, 'J')

In [44]:
point = utm.from_latlon(-24.9361993186,-51.3906013593)

In [45]:
point[0]

460564.002392906

In [46]:
dfUTM = pd.DataFrame([]) # initialize new dataframe
for rowIndex, row in dfGPS.iterrows(): #Iterating thru elements in dataframe
    point = utm.from_latlon(row['latitude'],row['longitude']) # convert to UTM coordinates
    pointUTM = pd.Series(dict(
                            x=point[0],
                            y=point[1],
                            z=row['elevation'],
                            zoneNumber=point[2],
                            zoneLetter=point[3]))
    dfUTM = dfUTM.append(pointUTM, ignore_index=True) # append newly converted data to dataframe

In [47]:
x = dfUTM[['x']].values
y = dfUTM[['y']].values
z = dfUTM[['z']].values
data = [
    go.Contour(
        z=z,
        x=x,
        y=y
    )]

py.iplot(data)